In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pickle,os
import trako as TKO

In [10]:
a = TKO.Util.loadvtp('/home/haehn/Dropbox/TRAKODATA/WholeBrain/RealTractDataIncludingTensorsAndScalars.vtk')

In [36]:
b = TKO.Util.loadvtp('/home/haehn/Dropbox/TRAKODATA/WholeBrain/RealTractDataIncludingTensorsAndScalars.vtk_restored_configqbi10.vtp')

In [12]:
a['scalar_names']

['FreeWater', 'tensor2', 'tensor1', 'EstimatedUncertainty']

In [37]:
b['scalar_names']

['FreeWater', 'tensor2', 'tensor1', 'EstimatedUncertainty']

In [56]:
b['scalars'][3].shape

(5650084,)

In [3]:
SUBJECT='/'
BLACKLIST = 'tracts_commissural'
DATAFOLDER = '/home/haehn/Dropbox/TRAKODATA/WholeBrain_VTK//'
# DATAFOLDER = '/home/haehn/Dropbox/TKOTEST/'
DATAFOLDER_TKO = DATAFOLDER[:-1]+'_TKO/'
DATAFOLDER_RESTORED = DATAFOLDER[:-1]+'_RESTORED/'
STATSFOLDER = DATAFOLDER[:-1]+'_STATS/'
DATAFOLDER += SUBJECT
DATAFOLDER_TKO += SUBJECT
DATAFOLDER_RESTORED += SUBJECT
STATSFILE = STATSFOLDER+SUBJECT[:-1]+'.p'

originalsize = []
compressedsize = []
number_of_streamlines = []
c_time = []
d_time = []


In [16]:
THRESHOLD = 10
scalarstats = {
    
}
propertystats = {
    
}
counter = 0
firstrun = True
t0 = time.time()
# for f in os.listdir(DATAFOLDER):

i_poly = a
r_poly = b

#     i_poly = TKO.Util.loadvtp(input)
i_scalars = i_poly['scalars']
i_nscalars = i_poly['scalar_names']
i_properties = i_poly['properties']
i_nproperties = i_poly['property_names']

#     r_poly = TKO.Util.loadvtp(restored)
r_scalars = r_poly['scalars']
r_properties = r_poly['properties']

#             print(i_poly['points'])
#             print(r_poly['points'])
#             print(i_scalars)
#             print(r_scalars)

if len(i_scalars)==0 or len(r_scalars) == 0:
    print('skipped', name)


#             if (np.isnan(np.any(i_scalars))):
#                 print('NaN', name, i_scalars)
#                 continue

#             if (np.isnan(np.any(r_scalars))):
#                 print('NaN2', name, i_scalars)
#                 continue

if firstrun:
    for i,s in enumerate(i_nscalars):
        scalarstats[s] = [0,0,0,0,np.inf,-np.inf]
#                     print ('reset')
    for i,p in enumerate(i_nproperties):
        propertystats[p] = [0,0,0,0,np.inf,-np.inf]

    firstrun = False



for i,s in enumerate(i_scalars):
    c_name = i_nscalars[i]

    stats = TKO.Util.error(i_scalars[i], r_scalars[i])
#                 print(stats)

#                 if (np.isnan(stats[0][0])):
#                     print(input, counter, r_scalars)

    scalarstats[c_name][0] += stats[0][0]
    scalarstats[c_name][1] += stats[0][1]
    scalarstats[c_name][2] += stats[0][2]
    scalarstats[c_name][3] += stats[0][3]
    scalarstats[c_name][4] = min(np.min(i_scalars[i]),scalarstats[c_name][4] )
    scalarstats[c_name][5] = max(np.max(i_scalars[i]),scalarstats[c_name][5] )

for i,s in enumerate(i_properties):
    c_name = i_nproperties[i]

    stats = TKO.Util.error(i_properties[i], r_properties[i])
#                 print(stats)

#                 if (np.isnan(stats[0][0])):
#                     print(input, counter, r_scalars)

    propertystats[c_name][0] += stats[0][0]
    propertystats[c_name][1] += stats[0][1]
    propertystats[c_name][2] += stats[0][2]
    propertystats[c_name][3] += stats[0][3]
    propertystats[c_name][4] = min(np.min(i_properties[i]),propertystats[c_name][4] )
    propertystats[c_name][5] = max(np.max(i_properties[i]),propertystats[c_name][5] )

counter += 1

            
# normalize
for s in scalarstats.keys():
    for i,v in enumerate(scalarstats[s]):
        if i > 3:
            continue
        scalarstats[s][i] /= counter
for s in propertystats.keys():
    for i,v in enumerate(propertystats[s]):
        if i > 3:
            continue
        propertystats[s][i] /= counter
print(time.time()-t0)

293.17223715782166


In [33]:
scalarstats

{'FreeWater': [0.0,
  3.0547380447387695e-05,
  1.4210282643034589e-05,
  9.33843148231972e-06,
  0.0,
  1.0],
 'tensor2': [1.5375793083194367e-08,
  4.924756353830162e-07,
  2.8949870056749205e-07,
  5.903690336594991e-08,
  -0.0013220137,
  0.0042674122],
 'tensor1': [2.1554678042434716e-08,
  3.8895998955013056e-07,
  2.2696508494846057e-07,
  4.628169492093548e-08,
  -0.0013191047,
  0.0030630874],
 'EstimatedUncertainty': [0.0,
  0.5986328125,
  0.29056376218795776,
  0.1768338829278946,
  0.03316625,
  19567.16]}

In [49]:
    mean= np.round(scalarstats['tensor2'][2],10),np.round(scalarstats['tensor2'][3],10)


In [50]:
mean

(2.895e-07, 5.9e-08)

In [52]:
prec = 10
for p in scalarstats.keys():
    ranger = np.round(scalarstats[p][4],prec),np.round(scalarstats[p][5],prec)
    mean= np.round(scalarstats[p][2],prec),np.round(scalarstats[p][3],prec)
    ranger_str = str(str(np.round(ranger[0],prec))+'-'+str(np.round(ranger[1],prec)))
    mean_str = str(mean[0])+'$\pm$'+str(mean[1])
    print(p, '($N$, range: ',ranger_str, ') \\\\')
for p in scalarstats.keys():
    ranger = np.round(scalarstats[p][4],prec),np.round(scalarstats[p][5],prec)
    mean= np.round(scalarstats[p][2],prec),np.round(scalarstats[p][3],prec)
    ranger_str = str(str(np.round(ranger[0],prec))+'-'+str(np.round(ranger[1],prec)))
    mean_str = str(mean[0])+'$\pm$'+str(mean[1])
    print(mean_str , '\\\\')

FreeWater ($N$, range:  0.0-1.0 ) \\
tensor2 ($N$, range:  -0.0013220137-0.0042674122 ) \\
tensor1 ($N$, range:  -0.0013191047-0.0030630874 ) \\
EstimatedUncertainty ($N$, range:  0.03316625-19567.16 ) \\
1.42103e-05$\pm$9.3384e-06 \\
2.895e-07$\pm$5.9e-08 \\
2.27e-07$\pm$4.63e-08 \\
0.2905637622$\pm$0.1768338829 \\


In [20]:
for p in propertystats.keys():
    ranger = np.round(propertystats[p][4],3),np.round(propertystats[p][5],3)
    mean= np.round(propertystats[p][2],3),np.round(propertystats[p][3],3)
    ranger_str = str(str(np.round(ranger[0],2))+'-'+str(np.round(ranger[1],2)))
    mean_str = str(mean[0])+'$\pm$'+str(mean[1])
    print(p, '($N$, range: ',ranger_str, ') \\\\')
for p in propertystats.keys():
    ranger = np.round(propertystats[p][4],3),np.round(propertystats[p][5],3)
    mean= np.round(propertystats[p][2],3),np.round(propertystats[p][3],3)
    ranger_str = str(str(np.round(ranger[0],2))+'-'+str(np.round(ranger[1],2)))
    mean_str = str(mean[0])+'$\pm$'+str(mean[1])
    print(mean_str , '\\\\')

In [27]:
compressedsize = 256311269
originalsize = 543022588

c_ratio = (1-float(compressedsize)/float(originalsize))*100
c_factor = float(originalsize) / float(compressedsize)

In [31]:
np.round(c_ratio,3)

52.799

In [32]:
np.round(c_factor,3)

2.119

In [30]:
np.round(compressedsize/1000000,2)

256.31